# Code examples for Simons (2023)

This notebook details the usage of the function `eigenvector_estimates_cis` applied to two examples. The first uses the built-in covariance matrix estimator based on the outer product and enforces homoskedasticity across columns of $M$. The example uses the `CovarianceMatrices` package to estimate a HAC-robust matrix.

## Inference for eigenvectors with homoskedastic noise

We have the model $M_{t} = M + \varepsilon_{t}$ and take the mean $\hat{M}$. We assume that the estimation error matrix $\hat{M} - M$ has $\text{i.i.d.}$ columns $ \left[\textbf{a}_{1} \, \textbf{a}_{2} \, \dots \, \textbf{a}_{p} \right] $ with finite covariance as per $\mathbb{E} \textbf{a}_{i} \textbf{a}_{j}^{\intercal} = \Omega \delta_{ij}$. Therefore, $\Psi = I_p \otimes \Omega$ as in the standard setup of the paper. We use 

In [7]:
include("eigenvector_estimates_cis.jl")
using Statistics, Random
M = [1 -3; -1 1];
T = 50;
p = size(M)[1];
q = 1;
r=p-q;
A = randn(MersenneTwister(1234),Float64,p,p,T) .+ M;
eigenvector_estimates_cis(A)

2×1×3 Array{Float64, 3}:
[:, :, 1] =
 -1.6473512926853844
  1.0

[:, :, 2] =
 -1.9056459093950218
  1.0

[:, :, 3] =
 -1.389056675975747
  1.0

Compare the above with the population analogue. Note that the first entry of the third array dimension is the point estimate, followed by the lower and upper bounds according to the critical value of $1.96$.

In [8]:
ev,R = eigen(M,sortby = x -> (-floor(real(x), digits = 6), floor(imag(x), digits = 6)))
R[1:r,1:q] * pinv(R[(r+1):p,1:q])

1×1 Matrix{Float64}:
 -1.732050807568877

## Regression on an intercept example with HAC covariance estimation

If we confront a case where we suspect that our error sequence is not homoskedastic, we can use a customised covariance matrix. For example, we may want to use a Newey-West-type robust covariance matrix.

In [9]:
using CovarianceMatrices
T=50;
M = [1 -3; -1 1];
p=size(M)[1]
A = randn(MersenneTwister(1234),Float64,2,2,T) .+ M;
vectorised_matrices = reshape(A,(p^2,T));
y = vectorised_matrices'
X   = ones(T,1)
_,K = size(X)
b   = X\y
res = y - X*b

kernel = QuadraticSpectralKernel{NeweyWest}()
bw = CovarianceMatrices.optimalbandwidth(kernel, res, prewhite=false);
Ψ   = lrvar(QuadraticSpectralKernel(bw), res, scale = T^2/(T-K))   # df adjustment is built into vcov
eigenvector_estimates_cis(A,1,1.96,false,Ψ)

2×1×3 Array{Float64, 3}:
[:, :, 1] =
 -1.6473512926853844
  1.0

[:, :, 2] =
 -3.381733547962595
  1.0

[:, :, 3] =
 0.08703096259182597
 1.0